In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb

In [2]:
def sort_blocks_last(d):
    return {**{k:v for k,v in d.items() if not k.endswith('blocks.gms')}, **d}

In [3]:
def apply_settings(gmodel):
    gmodel.groups = {k:k for k in gmodel.settings.get_conf('g_endo').union(gmodel.settings.get_conf('g_exo'))}
    [gmodel.settings.files.__setitem__(k,v) for k,v in gmodel.settings.functions.items()];
    write_to_db = [k for k,v in gmodel.settings.databases.items() if v == gmodel.database][0]
    with open(gmodel.settings.data_folder+'\\'+gmodel.settings.name+'_sets.gms',"w") as file:
        [file.writelines(c) for c in [gmodel.write_sets(), gmodel.write_aliased_sets(), gmodel.write_sets_other(), gmodel.write_sets_load(write_to_db), 
                                      gmodel.write_parameters(),gmodel.write_parameters_load(write_to_db)]];
    gmodel.settings.files[gmodel.settings.name+'_sets.gms'] = gmodel.settings.data_folder
    gmodel.settings.files.update(gmodel.settings.mfiles)
    with open(gmodel.settings.data_folder+'\\'+gmodel.settings.name+'_groupsload.gms', "w") as file:
        [file.writelines(c) for c in [gmodel.write_groups_load(write_to_db)]];
    gmodel.settings.files[gmodel.settings.name+'_groupsload.gms'] = gmodel.settings.data_folder

*Merge extra attributes:*

In [4]:
def merge_list_of_settings_extra(gsettings,ls):
    gsettings.functions = {k:v for s in ls for k,v in s.functions.items()}
    gsettings.mfiles = sort_blocks_last({k:v for s in ls for k,v in s.mfiles.items()})

## Test various version of technology module

## 1: Logit

### 1.1 Database

In [5]:
db = DataBase.GPM_database()

Technology parameters:

In [6]:
db['i'] = pd.Index(range(11), name='i')
db['theta'] = pd.Series([0.2]*len(db['i']),index = db.get('i'), name = 'theta')
db['p'] = pd.Series(np.linspace(0, 10, len(db['i'])), index = db.get('i'), name = 'p')
db['sigma'] = 1

Initial values, endogenous variables:

In [7]:
db['tau'] = pd.Series(0.5, index = db.get('i'), name = 'tau')
db['pe'] = 5
db['lambda'] = 5

### 1.2 Gams model

*Version 1:*

In [8]:
name = 'V1'
work_folder = directory['work']
gams_folder = directory['gams']+'\\Test_V1'
g_endo = ['G_endo'];
blocks = ['B_V1'];
g_exo = ['G_exo'];
files = {k: gams_folder for k in ('V1_groups.gms','V1_blocks.gms')}
functions = {'functions.gms': gams_folder}

*Initialize settings object, and add settings:*

In [9]:
gsettings = DB2Gams.gams_settings(work_folder=work_folder,**{'name':name,'data_folder': work_folder})
gsettings.get_conf('g_endo').update(g_endo)
gsettings.get_conf('g_exo').update(g_exo)
gsettings.get_conf('blocks').update(blocks)
gsettings.functions = functions
gsettings.mfiles = files
gmodel = DB2Gams.gams_model_py(gsettings=gsettings)

*Add data:*

In [10]:
gsettings.add_database(db) # add data
gmodel.database.merge_internal() # write to a gdx file in the working directory

*apply standard settings:*

In [11]:
apply_settings(gmodel)

*Initialize model and run:*

In [12]:
g = DB2Gams.gams_model(gsettings = gsettings)
g.run()
print(g.modelstat,g.solvestat)

16.0 1.0


*Version 2:*

In [18]:
name = 'V2'
work_folder = directory['work']
gams_folder = directory['gams']+'\\Test_V1'
g_endo = ['G_endo'];
blocks = ['B_V2'];
g_exo = ['G_exo'];
files = {k: gams_folder for k in ('V1_groups.gms','V1_blocks.gms')}
functions = {'functions.gms': gams_folder}

*Initialize settings object, and add settings:*

In [20]:
gs2 = DB2Gams.gams_settings(work_folder = gsettings.ws,**{'name':name,'data_folder': work_folder})
gs2.get_conf('g_endo').update(g_endo)
gs2.get_conf('g_exo').update(g_exo)
gs2.get_conf('blocks').update(blocks)
gs2.functions = functions
gs2.mfiles = files
gm2 = DB2Gams.gams_model_py(gsettings=gs2)

*Add data:*

In [21]:
gs2.add_database(db) # add data
gm2.database.merge_internal() # write to a gdx file in the working directory

*apply standard settings:*

In [22]:
apply_settings(gm2)

*Initialize model and run:*

In [23]:
g2 = DB2Gams.gams_model(gsettings = gs2)
g2.run()
print(g2.modelstat,g2.solvestat)

16.0 1.0


*Compare versions 1,2*

In [33]:
g.out_db.get('pe')-g2.out_db.get('pe')

1.3260956818201475